In [38]:
import pandas as pd
import numpy as np
from datetime import datetime


In [39]:

# 1. Cargar los datos (asumiendo que ya tienes un DataFrame)
# Si necesitas cargar desde Excel:
df = pd.read_excel('raw.xlsx', sheet_name="Hoja1")


In [40]:
meses_es_to_en = {
    'ene': '01', 'feb': '02', 'mar': '03', 'abr': '04', 'may': '05', 'jun': '06',
    'jul': '07', 'ago': '08', 'sep': '09', 'oct': '10', 'nov': '11', 'dic': '12'
}

def convertir_fecha(fecha_str):
    try:
        fecha_str = fecha_str.replace("\xa0", "")
        dia, mes_es, anio = fecha_str.split('-')
        
        mes_en = meses_es_to_en[mes_es]
        return datetime.strptime(f"{dia}-{mes_en}-{anio}", "%d-%m-%Y")
    except Exception as e:
        print(f"Error convirtiendo fecha {fecha_str}: {e}")
        return pd.NaT

df['Vigente a contar de'] = df['Vigente a contar de'].apply(convertir_fecha)

In [41]:
# Convertir vigencia a datetime para ordenamiento

df['Vigente a contar de'] = pd.to_datetime(df['Vigente a contar de'], format='%Y-%m-01')

df["key"] = df["NOMBRE CORTO"] + "," + df["Localidad/sector"]
df.drop(columns=["NOMBRE CORTO", "Localidad/sector"])


,Vigente a contar de,CARGO FIJO,AP NO PUNTA,AP PUNTA,AP SOBRECONSUMO,key
0,2025-02-04,2044,"703,91",0,0,"MELIPILLA NORTE,VILLA LOS EDUCADORES"
1,2023-12-18,1992,"687,38",0,0,"MELIPILLA NORTE,VILLA LOS EDUCADORES"
2,2022-11-15,1886,"672,59",0,0,"MELIPILLA NORTE,VILLA LOS EDUCADORES"
3,2022-08-15,1823,"646,36",0,0,"MELIPILLA NORTE,VILLA LOS EDUCADORES"
4,2022-03-10,1723,"602,98",0,0,"MELIPILLA NORTE,VILLA LOS EDUCADORES"
...,...,...,...,...,...,...
165,2007-10-31,1137,"553,75","545,83","1183,7","SANTA ROSA DEL PERAL,SANTA ROSA DEL PERAL"
166,2006-01-15,972,"478,95","471,82","1023,27","SANTA ROSA DEL PERAL,SANTA ROSA DEL PERAL"
167,2005-11-15,1007,"493,49","486,27","1054,3","SANTA ROSA DEL PERAL,SANTA ROSA DEL PERAL"
168,2004-01-15,1854,"448,15","369,34","735,8","SANTA ROSA DEL PERAL,SANTA ROSA DEL PERAL"


In [42]:
# 2. Ordenar por empresa y vigencia (ascendente)
df = df.sort_values(['key', 'Vigente a contar de'])

# 3. Crear rango de fechas completo
start_date = '2012-01'
end_date = datetime.now().strftime('%Y-%m-01')  # Hasta el mes actual
date_range = pd.date_range(start=start_date, end=end_date, freq='MS')  # MS = Month Start

# 4. Función para expandir cada empresa
def expand_empresa(group):
    # Crear DataFrame con todas las fechas
    all_dates = pd.DataFrame({'fecha': date_range})
    
    # Hacer merge asof para asignar la tarifa vigente más reciente
    merged = pd.merge_asof(
        all_dates,
        group.rename(columns={'Vigente a contar de': 'fecha'}),
        left_on='fecha',
        right_on='fecha',
        direction='backward'
    )
    
    # Llenar NA (para periodos antes de la primera vigencia)
    merged['key'] = group['key'].iloc[0]
    merged['CARGO FIJO'] = merged['CARGO FIJO'].str.replace(' ', '').str.replace(',', '.').astype(float).fillna(method='bfill')
    merged['AP NO PUNTA'] = merged['AP NO PUNTA'].str.replace(' ', '').str.replace(',', '.').astype(float).fillna(method='bfill')
    merged['AP PUNTA'] = merged['AP PUNTA'].str.replace(' ', '').str.replace(',', '.').astype(float).fillna(method='bfill')
    merged['AP SOBRECONSUMO'] = merged['AP SOBRECONSUMO'].str.replace(' ', '').str.replace(',', '.').astype(float).fillna(method='bfill')
    
    return merged[['fecha', 'key', 'CARGO FIJO', "AP NO PUNTA", "AP PUNTA", "AP SOBRECONSUMO"]]

# 5. Aplicar a cada empresa
resultado = df.groupby('key', group_keys=False).apply(expand_empresa)

# 6. Formatear fecha como AAA-MM
resultado['fecha'] = resultado['fecha'].dt.strftime('%Y-%m-01')

# Mostrar resultado
resultado["key"] = resultado["key"].str.rstrip()

# Guardar en Excel si es necesario
resultado.to_excel('tarifas_historicas.xlsx')

C:\Users\IanLagreze\AppData\Local\Temp\ipykernel_33336\2486495450.py:25: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged['CARGO FIJO'] = merged['CARGO FIJO'].str.replace(' ', '').str.replace(',', '.').astype(float).fillna(method='bfill')
C:\Users\IanLagreze\AppData\Local\Temp\ipykernel_33336\2486495450.py:26: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged['AP NO PUNTA'] = merged['AP NO PUNTA'].str.replace(' ', '').str.replace(',', '.').astype(float).fillna(method='bfill')
C:\Users\IanLagreze\AppData\Local\Temp\ipykernel_33336\2486495450.py:27: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged['AP PUNTA'] = merged['AP PUNTA'].str.replace(' ', '').str.replace(',', '.').astype(float).fillna(method='bfill')
C:\Use